In [3]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
file = ".\\data"+ "\\phaseone-tradedata-C-november.parquet"

df = pq.read_table(file).to_pandas()

df.replace({'Electrical equipment and mac.hioery':'Electrical equipment and machinery',
           'Industrial mac.hin ery': 'Industrial machinery'}, inplace = True)

df["hs2"] = df.hs4.str[0:2]

In [5]:
df = df[df["high_catagory"] != "not in aggreement"]

In [6]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

end_use = "hs?get=E_COMMODITY,ALL_VAL_YR,E_COMMODITY_SDESC"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time=2019" + "&COMM_LVL=HS2"

url = url + "&CTY_CODE=5700"

r = requests.get(url) 
    
print(r)
    
dfc = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

dfc.columns = r.json()[0]

dfc = dfc.groupby("E_COMMODITY").agg({"E_COMMODITY_SDESC":"first"})

<Response [200]>


In [7]:
dfc.head()

,E_COMMODITY_SDESC
E_COMMODITY,
01,LIVE ANIMALS
02,MEAT AND EDIBLE MEAT OFFAL
03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES"
04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...
05,"PRODUCTS OF ANIMAL ORIGIN, NESOI"


In [8]:
df.head()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge,hs2
0,2013-01-01,5700,4806.0,841090,0.0,-,8410,8410,"Hydraulic turbines, water wheels and regulator...",Industrial machinery,1. Manufactured Goods,both,84
11,2013-01-01,5700,279107.0,900211,0.0,-,9002,9002,"Lens,es prism, s mirrors and other optical e...",Optical and medical instruments,1. Manufactured Goods,both,90
26,2013-01-01,5700,4007.0,110630,0.0,-,1106,1106,Flour and meal of dried leguminous vegetables ...,Cereals,2. Agriculture,both,11
27,2013-01-01,5700,404107.0,130212,0.0,-,1302,1302,Vegetable saps and extracts; pectic substances...,Other agricultural commodities,2. Agriculture,both,13
28,2013-01-01,5700,438996.0,841191,0.0,-,8411,8411,"Turboj e,ts turbopropellers and other gas ...",Industrial machinery,1. Manufactured Goods,both,84


In [9]:
aws = df.groupby(["hs2","time"]).agg({"china_exports":"sum"})

In [10]:
aws = aws.merge(dfc, left_on = ["hs2"], right_index = True, how = "left")

aws.reset_index(inplace = True)

aws["E_COMMODITY_SDESC"] = aws["E_COMMODITY_SDESC"] + ", HS CODE " + aws.hs2

In [11]:
aws.set_index(["E_COMMODITY_SDESC","time"],inplace = True)

In [12]:
out_file = ".\\data"+ "\\data.parquet"

pq.write_table(pa.Table.from_pandas(aws), out_file)

In [13]:
file = ".\\data"+ "\\data.parquet"

df = pq.read_table(file).to_pandas()

df.loc["IRON AND STEEL, HS CODE 72"]

,hs2,china_exports
time,,
2013-01-01,72,114103112.0
2013-02-01,72,158804662.0
2013-03-01,72,128598274.0
2013-04-01,72,98804647.0
2013-05-01,72,146780917.0
...,...,...
2020-07-01,72,40085617.0
2020-08-01,72,57228239.0
2020-09-01,72,34440581.0


In [14]:
df.china_exports.max()

3557751166.0

In [15]:
df[df.hs2 == "31"]

,,hs2,china_exports
E_COMMODITY_SDESC,time,,
